In [ ]:
import xarray as xr
import numpy as np
import os

# Base directory where your per-variable subfolders live
base_dir = r"D:\ERA5_Modified"

# Template for each variable’s yearly file
file_template = {
    "ssr": os.path.join(base_dir, "net_ssr", "ssr_{year}.nc"),
    "str":  os.path.join(base_dir, "net_lwr", "str_{year}.nc"),
    "sp":   os.path.join(base_dir, "surface_pr", "sp_{year}.nc"),
    "d2m":  os.path.join(base_dir, "Tdewm",      "d2m_mean_{year}.nc"),
    "tmax": os.path.join(base_dir, "Tmax",        "t2m_max_{year}.nc"),
    "tmin": os.path.join(base_dir, "Tmin",        "t2m_min_{year}.nc"),
    "u10":  os.path.join(base_dir, "U10",         "u10_{year}.nc"),
    "v10":  os.path.join(base_dir, "V10",         "v10_{year}.nc")
}

# Output directory for per-year ET0/Rn files
out_dir = os.path.join(base_dir, "ET0_Rn_per_year_PET")
os.makedirs(out_dir, exist_ok=True)

for year in range(1941, 2024):
    print(f"Processing {year}…")

    # 1) Load each variable’s dataset for this year
    datasets = {
        var: xr.open_dataset(path.format(year=year))
        for var, path in file_template.items()
    }
    # extract the single DataArray from each
    dv = {v: ds[list(ds.data_vars)[0]] for v, ds in datasets.items()}

    # 2) Compute intermediate fields
    # temperatures (K → °C)
    tmax = dv["tmax"] - 273.15
    tmin = dv["tmin"] - 273.15
    tdew = dv["d2m" ] - 273.15
    tmean = (tmax + tmin) / 2

    # radiation (J/m²/h → MJ/m²/day)
    ssr = dv["ssr"] * 24 / 1e6
    str = dv["str"] * 24 / 1e6

    # net radiation
    rn  = ssr + str

    # slope of saturation vapour pressure curve Δ [kPa/°C]
    delta = (
        4098 * (0.6108 * np.exp((17.27 * tmean) / (tmean + 237.3)))
        / (tmean + 237.3)**2
    )

    # saturation vapour pressure eₛ and actual vapour pressure eₐ
    es = (
        0.6108 * np.exp((17.27 * tmax) / (tmax + 237.3))
      + 0.6108 * np.exp((17.27 * tmin) / (tmin + 237.3))
    ) / 2
    ea = 0.6108 * np.exp((17.27 * tdew) / (tdew + 237.3))

    # wind speed at 2 m
    u10 = dv["u10"]
    v10 = dv["v10"]
    u2  = 0.748 * np.sqrt(u10**2 + v10**2)

    # psychrometric constant γ [kPa/°C]
    sp_kpa = dv["sp"] / 1000
    gamma  = 0.000665 * sp_kpa

    # 3) FAO-56 Penman-Monteith ET₀ [mm/day]
    num = (0.408 * delta * rn) + (gamma * (900/(tmean+273)) * u2 * (es - ea))
    den = delta + gamma * (1 + 0.34 * u2)
    et0 = num / den
    et0 = et0.where(et0 >= 0)  # mask negatives

    # 4) Build per-year Dataset
    ds_out = xr.Dataset(
        {
            "ET0": et0.assign_attrs({
                "units": "mm/day",
                "long_name": "Reference Evapotranspiration (FAO-56 PM)"
            }),
            "Rn":  rn.assign_attrs({
                "units": "MJ/m²/day",
                "long_name": "Net Radiation"
            })
        },
        coords={
            # reuse each variable’s coords (they all share the same)
            "time": dv["tmax"].coords["valid_time"],
            "latitude": dv["tmax"].coords["latitude"],
            "longitude": dv["tmax"].coords["longitude"]
        },
        attrs={
            "title": f"Daily ET₀ & Net Radiation, {year}",
            "source": "ERA5 + FAO-56 Penman-Monteith"
        }
    )

    # 5) Save to NetCDF
    out_path = os.path.join(out_dir, f"ET0_Rn_{year}_PET.nc")
    ds_out.to_netcdf(out_path)
    print(f" → wrote {out_path}")

print("All years processed.")

In [ ]:
# ====== LEAN CONFIG ======
PATH_P = r"D:\ERA5_Modified\Tp_1941-2023.nc"   # ERA5 daily precip (tp) in meters
ALPHA = 0.05
MIN_YEARS = 20   # require at least this many complete Oct–Mar seasons

# ====== IMPORTS ======
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pymannkendall as pmk

# ====== LOAD & MONTHLY SUM ======
# Daily -> mm, then monthly totals (mm/month)
ds = xr.open_dataset(PATH_P, engine="netcdf4")
P_daily_mm = ds["tp"] * 1000.0
mon = P_daily_mm.resample(valid_time="MS").sum(skipna=True)

# ====== SELECT OCT–MAR & ASSIGN WATER YEAR (explicit & readable) ======
month = mon["valid_time"].dt.month
is_cool = month.isin([10, 11, 12, 1, 2, 3])
mon_cs = mon.sel(valid_time=is_cool)

year = mon_cs["valid_time"].dt.year
wy = xr.where(mon_cs["valid_time"].dt.month >= 10, year + 1, year)  # Oct–Dec -> next year
mon_cs = mon_cs.assign_coords(wy=("valid_time", wy.data))

# ====== AGGREGATE TO COOL-SEASON TOTALS PER WY (require all 6 months) ======
cs_wy = mon_cs.groupby("wy").sum("valid_time", skipna=True, min_count=6)
cs_wy = cs_wy.rename({"wy": "water_year"}).sortby("water_year")  # (water_year, lat, lon)
# Ensure MK core dimension (water_year) is one chunk
cs_wy = cs_wy.chunk({"water_year": -1})

# ====== REGULAR MK + THEIL–SEN (per grid cell, vectorized) ======
def mk_wrapper(ts_1d):
    """Regular MK on a 1D Oct–Mar seasonal series. Returns (Z, p, slope)."""
    x = np.asarray(ts_1d, float)
    x = x[np.isfinite(x)]  # drop NaNs (incomplete seasons)
    if x.size < MIN_YEARS:
        return np.nan, np.nan, np.nan
    r = pmk.original_test(x)  # returns z, p, slope, etc.
    return float(r.z), float(r.p), float(r.slope)

ZPS = xr.apply_ufunc(
    mk_wrapper,
    cs_wy,
    input_core_dims=[["water_year"]],
    output_core_dims=[[], [], []],   # Z, p, slope
    vectorize=True,
    dask="parallelized",
    output_dtypes=[float, float, float],
)

Z = ZPS[0].rename("Z_mk_OctMar")
P = ZPS[1].rename("p_mk_OctMar")
SLOPE = ZPS[2].rename("sen_slope_OctMar")  # units: mm/season per year

SIG = (P < ALPHA)
Z_masked = Z.where(SIG)
# --- convert mm/season/year -> %/year
season_mean = cs_wy.mean("water_year")                     # mm/season
season_mean = season_mean.where(season_mean > 0)           # avoid /0 in arid cells

SLOPE_pct = (SLOPE / season_mean) * 100.0                  # %/year
SLOPE_pct = SLOPE_pct.rename("sen_slope_pct_OctMar")

# mask by significance (same mask as before)
SLOPE_pct_masked = SLOPE_pct.where(SIG)
#SLOPE_masked = SLOPE.where(SIG)

# ====== PLOTS (side-by-side) ======
proj = ccrs.PlateCarree()
fig, axes = plt.subplots(
    1, 3, figsize=(15, 5), subplot_kw={'projection': proj}, constrained_layout=True
)

plots = [
    (Z, "Regular MK Z P — Cool Season (Oct–Mar)"),
    (Z_masked, f"Regular MK Z P (p < {ALPHA}) — Cool Season (Oct–Mar)"),
    (SLOPE_pct_masked, f"Theil–Sen slope P (p < {ALPHA}) — Cool Season (Oct–Mar)"),
]

for ax, (da, title) in zip(axes, plots):
    ax.coastlines(linewidth=0.8)
    da.plot.pcolormesh(
        ax=ax, transform=proj, add_colorbar=True, cbar_kwargs={"shrink": 0.7}
    )
    ax.set_title(title)
# ====== HUC2 OUTLINE ONLY (robust with Cartopy) ======
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon

SHP_BASINS = r"D:\OneDrive\UCLA_F24\4.Sum25\Whiplash\2.Data_In\HUC2_Boundaries\HU02_WUS.shp"
basins = gpd.read_file(SHP_BASINS)

# Print to confirm
print("HUC2 CRS:", basins.crs)
print("bounds (minx, miny, maxx, maxy):", basins.total_bounds)

# If not lon/lat, reproject to EPSG:4326 for PlateCarree
if basins.crs is None:
    basins = basins.set_crs(epsg=4326)  # assume lon/lat if missing
elif basins.crs.to_epsg() != 4326:
    basins = basins.to_crs(epsg=4326)

# (Optional) clean invalid geoms
basins["geometry"] = basins.geometry.buffer(0)

# Draw ONLY on the last subplot
ax_last = axes[-1]

for geom in basins.geometry:
    if geom is None:
        continue
    parts = list(geom.geoms) if isinstance(geom, MultiPolygon) else [geom]
    ax_last.add_geometries(
        parts,
        crs=ccrs.PlateCarree(),   # data are in lon/lat now
        facecolor="none",
        edgecolor="black",
        linewidth=0.8,            # bump up from 0.1 to be visible
        zorder=30,
    )

plt.show()

In [ ]:
# ====== LEAN CONFIG ======
PATH_P = r"D:\ERA5_Modified\ET0_Rn_1941-2023_PET.nc"  
ALPHA = 0.05
MIN_YEARS = 20   # require at least this many complete Oct–Mar seasons

# ====== IMPORTS ======
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pymannkendall as pmk
import dask

# ====== LOAD & MONTHLY SUM ======
ds = xr.open_dataset(PATH_P, engine="netcdf4")
P_daily_mm = ds["ET0"]
mon = P_daily_mm.resample(valid_time="MS").sum(skipna=True)

# ====== SELECT OCT–MAR & ASSIGN WATER YEAR (explicit & readable) ======
month = mon["valid_time"].dt.month
is_warm = month.isin([4, 5, 6, 7, 8, 9])
mon_cs = mon.sel(valid_time=is_warm)

year = mon_cs["valid_time"].dt.year
mon_cs = mon_cs.assign_coords(wy=year)

# ====== AGGREGATE TO COOL-SEASON TOTALS PER WY (require all 6 months) ======
cs_wy = mon_cs.groupby("wy").sum("valid_time", skipna=True, min_count=6)
cs_wy = cs_wy.rename({"wy": "water_year"}).sortby("water_year")  # (water_year, lat, lon)
# Ensure MK core dimension (water_year) is one chunk
cs_wy = cs_wy.chunk({"water_year": -1})

# ====== REGULAR MK + THEIL–SEN (per grid cell, vectorized) ======
def mk_wrapper(ts_1d):
    """Regular MK on a 1D Oct–Mar seasonal series. Returns (Z, p, slope)."""
    x = np.asarray(ts_1d, float)
    x = x[np.isfinite(x)]  # drop NaNs (incomplete seasons)
    if x.size < MIN_YEARS:
        return np.nan, np.nan, np.nan
    r = pmk.original_test(x)  # returns z, p, slope, etc.
    return float(r.z), float(r.p), float(r.slope)

ZPS = xr.apply_ufunc(
    mk_wrapper,
    cs_wy,
    input_core_dims=[["water_year"]],
    output_core_dims=[[], [], []],   # Z, p, slope
    vectorize=True,
    dask="parallelized",
    output_dtypes=[float, float, float],
)

Z = ZPS[0].rename("Z_mk_AprSep")
P = ZPS[1].rename("p_mk_AprSep")
SLOPE = ZPS[2].rename("sen_slope_AprSep")

SIG = (P < ALPHA)
Z_masked = Z.where(SIG)
# --- convert mm/season/year -> %/year
season_mean = cs_wy.mean("water_year")                     # mm/season
season_mean = season_mean.where(season_mean > 0)           # avoid /0 in arid cells

SLOPE_pct = (SLOPE / season_mean) * 100.0                  # %/year
SLOPE_pct = SLOPE_pct.rename("sen_slope_pct_OctMar")

# mask by significance (same mask as before)
SLOPE_pct_masked = SLOPE_pct.where(SIG)
#SLOPE_masked = SLOPE.where(SIG)

# ====== HUC2 OUTLINE ONLY (robust with Cartopy) ======BASINS LOAD
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon

SHP_BASINS = r"D:\OneDrive\UCLA_F24\4.Sum25\Whiplash\2.Data_In\HUC2_Boundaries\HU02_WUS.shp"
basins = gpd.read_file(SHP_BASINS)

# Print to confirm
print("HUC2 CRS:", basins.crs)
print("bounds (minx, miny, maxx, maxy):", basins.total_bounds)

# If not lon/lat, reproject to EPSG:4326 for PlateCarree
if basins.crs is None:
    basins = basins.set_crs(epsg=4326)  # assume lon/lat if missing
elif basins.crs.to_epsg() != 4326:
    basins = basins.to_crs(epsg=4326)

# (Optional) clean invalid geoms
basins["geometry"] = basins.geometry.buffer(0)

# ====== PER-BASIN COUNTS (optimized single-pass) ======
import pandas as pd, regionmask

# Build region definitions
regions = regionmask.Regions(
    outlines=basins.geometry.values,
    names=basins["name"].astype(str).tolist(),
    numbers=basins["huc2"].astype(int).tolist(),
)

# Build region mask on your grid (use positional args for older regionmask)
rmask = regions.mask(Z["longitude"].values, Z["latitude"].values)
if not isinstance(rmask, xr.DataArray):
    rmask = xr.DataArray(rmask, dims=("lat","lon"))
rmask = rmask.rename({"lat":"latitude","lon":"longitude"}).assign_coords(
    latitude=Z["latitude"], longitude=Z["longitude"]
)

# Optional: persist arrays reused multiple times (helps on larger domains)
#valid, SIG, SLOPE, rmask = xr.persist(season_mean.notnull(), SIG, SLOPE, rmask)

# Build boolean masks ONCE and cast to small ints for fast summation
valid_i = season_mean.notnull().fillna(False).astype("int8")
sig_i   = (valid_i.astype(bool) & SIG).fillna(False).astype("int8")
pos_i   = (valid_i.astype(bool) & SIG & (SLOPE > 0)).fillna(False).astype("int8")
neg_i   = (valid_i.astype(bool) & SIG & (SLOPE < 0)).fillna(False).astype("int8")

# Label array (region ids) aligned to grid
lab = rmask.rename("region").astype("int16")

# Stack to 1D and reduce by region label ONCE
lab1   = lab.stack(pt=("latitude","longitude"))
tot_g  = valid_i.stack(pt=("latitude","longitude")).groupby(lab1).sum()
sig_g  = sig_i.stack(pt=("latitude","longitude")).groupby(lab1).sum()
pos_g  = pos_i.stack(pt=("latitude","longitude")).groupby(lab1).sum()
neg_g  = neg_i.stack(pt=("latitude","longitude")).groupby(lab1).sum()

# Compute all aggregates in one pass
tot_g, sig_g, pos_g, neg_g = dask.compute(tot_g, sig_g, pos_g, neg_g)

# Assemble tidy table
rows = []
for num, name in zip(regions.numbers, regions.names):
    if num in tot_g.indexes["region"]:
        tot = int(tot_g.sel(region=num).item())
        sig = int(sig_g.sel(region=num).item())
        pos = int(pos_g.sel(region=num).item())
        neg = int(neg_g.sel(region=num).item())
        rows.append({
            "huc2": num, "basin": name, "total_cells": tot,
            "sig_total": sig, "sig_pos": pos, "sig_neg": neg,
            "sig_%": (100*sig/tot) if tot else 0.0
        })
summary = pd.DataFrame(rows).sort_values("huc2")
print(summary.to_string(index=False))

# ====== PLOTS (side-by-side) ======
proj = ccrs.PlateCarree()
fig, axes = plt.subplots(
    1, 3, figsize=(15, 5), subplot_kw={'projection': proj}, constrained_layout=True
)

plots = [
    (Z, "Regular MK Z ET0 — Warm Season (Apr–Sep)"),
    (Z_masked, f"Regular MK Z ET0 (p < {ALPHA}) — Warm Season (Apr–Sep)"),
    (SLOPE_pct_masked, f"Theil–Sen slope ET0 (p < {ALPHA}) — Warm Season (Apr–Sep)"),
]

for ax, (da, title) in zip(axes, plots):
    ax.coastlines(linewidth=0.8)
    da.plot.pcolormesh(
        ax=ax, transform=proj, add_colorbar=True, cbar_kwargs={"shrink": 0.7}
    )
    ax.set_title(title)

# ====== HUC2 OUTLINE ONLY (robust with Cartopy) ======
# explicitly pick the two you want
ax_last = [axes[-2], axes[-1]]   # this is a list of the last two axes

for ax in ax_last:
    for geom in basins.geometry:
        if geom is None:
            continue
        parts = list(geom.geoms) if isinstance(geom, MultiPolygon) else [geom]
        ax.add_geometries(
        parts,
        crs=ccrs.PlateCarree(),   # data are in lon/lat now
        facecolor="none",
        edgecolor="black",
        linewidth=0.8,
        zorder=30,
    )

plt.show()